In [ ]:
''' 
for each year - merge df with text content with company info

stack all years

remove reports from companies from times when they were not in S&P 500 

lemmatize text

'''

In [1]:
import pickle
import pandas as pd
import numpy as np
import datetime
from nltk.stem import WordNetLemmatizer

In [2]:
all_text_data = pd.DataFrame()

In [3]:
for year in [2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011]:
    
    # df of paragraphs about risk
    with open('../data/preproc/preproc_text_' + str(year) + '.pickle', 'rb') as read_file:
        prepped_text = pickle.load(read_file)
      
    # list with company info
    sp500_list = pd.read_pickle('../data/urls_' + str(year) + '.pickle')
    
    # merge 
    text_df_sp = pd.merge(prepped_text, sp500_list, how="left", on=["ticker", "linkToTxt"])
        
    #stack     
    all_text_data = pd.concat([all_text_data, text_df_sp], axis=0)

print(all_text_data.shape)
    

(396413, 16)


In [4]:
all_text_data.head()

,value,ticker,linkToTxt,preproc_text,id,accessionNo,cik,companyName,companyNameLong,formType,filedAt,linkToHtml,irsNo,fiscalYearEnd,sic,filed_at
0,We produce and distribute high-quality video c...,T,https://www.sec.gov/Archives/edgar/data/732717...,We produce and distribute high quality video c...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,2020-02-19T21:42:50-05:00,https://www.sec.gov/Archives/edgar/data/732717...,431301883,1231,4813 Telephone Communications (No Radiotelephone),2020-02-19 21:42:50-05:00
1,Our costs to provide current benefits and fund...,T,https://www.sec.gov/Archives/edgar/data/732717...,Our costs to provide current benefits and fund...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,2020-02-19T21:42:50-05:00,https://www.sec.gov/Archives/edgar/data/732717...,431301883,1231,4813 Telephone Communications (No Radiotelephone),2020-02-19 21:42:50-05:00
2,Our international operations have increased ou...,T,https://www.sec.gov/Archives/edgar/data/732717...,Our international operations have increased ou...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,2020-02-19T21:42:50-05:00,https://www.sec.gov/Archives/edgar/data/732717...,431301883,1231,4813 Telephone Communications (No Radiotelephone),2020-02-19 21:42:50-05:00
3,"In addition, operating in foreign countries al...",T,https://www.sec.gov/Archives/edgar/data/732717...,In addition operating in foreign countries al...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,2020-02-19T21:42:50-05:00,https://www.sec.gov/Archives/edgar/data/732717...,431301883,1231,4813 Telephone Communications (No Radiotelephone),2020-02-19 21:42:50-05:00
4,Our video subsidiaries derive substantial reve...,T,https://www.sec.gov/Archives/edgar/data/732717...,Our video subsidiaries derive substantial reve...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,2020-02-19T21:42:50-05:00,https://www.sec.gov/Archives/edgar/data/732717...,431301883,1231,4813 Telephone Communications (No Radiotelephone),2020-02-19 21:42:50-05:00


In [ ]:
# drop rows for companies that were not in S&P at time of annual report

In [5]:
# remove 3 added in 2021
f1_all = all_text_data[~all_text_data.ticker.isin(['MPWR', 'TRMB', 'ENPH'])]
f1_all.shape

(395167, 16)

In [6]:
# list of added tickers 
sp_adds = pd.read_excel('../sp500_changes.xlsx', sheet_name='added')
sp_adds.head()

,Date_Added,Added_Ticker,Added_Security
0,2020-12-21,TSLA,"Tesla, Inc."
1,2020-10-09,VNT,Vontier
2,2020-10-07,POOL,Pool Corporation
3,2020-09-21,ETSY,Etsy
4,2020-09-21,TER,Teradyne


In [7]:
sp_adds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date_Added      226 non-null    datetime64[ns]
 1   Added_Ticker    226 non-null    object        
 2   Added_Security  226 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 5.4+ KB


In [8]:
# check for duplicates
sp_adds.Added_Ticker.duplicated().value_counts()

False    226
Name: Added_Ticker, dtype: int64

In [9]:
# list of removed tickers
sp_removed = pd.read_excel('../sp500_changes.xlsx', sheet_name='removed')
sp_removed.head()

,Date_Removed,Removed_Ticker,Removed_Security
0,2017-06-19,YHOO,Yahoo! Inc.
1,2010-06-28,XTO,XTO Energy Inc.
2,2018-09-14,XL,XL Group
3,2020-03-03,XEC,Cimarex Energy
4,2014-07-02,X,United States Steel Corporation


In [10]:
# check for duplicates
sp_removed.Removed_Ticker.duplicated().value_counts()

False    219
Name: Removed_Ticker, dtype: int64

In [11]:
# merge add and removed dates with df
# flag rows to drop

print(f1_all.shape)

merge1 = pd.merge(f1_all, sp_adds, how='left', left_on='ticker', right_on='Added_Ticker')
print(merge1.shape)

merge2 = pd.merge(merge1, sp_removed, how='left', left_on='ticker', right_on='Removed_Ticker')
print(merge2.shape)

(395167, 16)
(395167, 19)
(395167, 22)


In [12]:
merge2.head(2)

,value,ticker,linkToTxt,preproc_text,id,accessionNo,cik,companyName,companyNameLong,formType,...,irsNo,fiscalYearEnd,sic,filed_at,Date_Added,Added_Ticker,Added_Security,Date_Removed,Removed_Ticker,Removed_Security
0,We produce and distribute high-quality video c...,T,https://www.sec.gov/Archives/edgar/data/732717...,We produce and distribute high quality video c...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,...,431301883,1231,4813 Telephone Communications (No Radiotelephone),2020-02-19 21:42:50-05:00,NaT,NaN,NaN,NaT,NaN,NaN
1,Our costs to provide current benefits and fund...,T,https://www.sec.gov/Archives/edgar/data/732717...,Our costs to provide current benefits and fund...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,...,431301883,1231,4813 Telephone Communications (No Radiotelephone),2020-02-19 21:42:50-05:00,NaT,NaN,NaN,NaT,NaN,NaN


In [13]:
merge2['date_filed_str'] = merge2['filedAt'].str[:10]
merge2['date_filed'] = pd.to_datetime(merge2['date_filed_str'], format='%Y-%m-%d')

In [ ]:
#merge2.info()

In [14]:
merge2.head(2)

,value,ticker,linkToTxt,preproc_text,id,accessionNo,cik,companyName,companyNameLong,formType,...,sic,filed_at,Date_Added,Added_Ticker,Added_Security,Date_Removed,Removed_Ticker,Removed_Security,date_filed_str,date_filed
0,We produce and distribute high-quality video c...,T,https://www.sec.gov/Archives/edgar/data/732717...,We produce and distribute high quality video c...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,...,4813 Telephone Communications (No Radiotelephone),2020-02-19 21:42:50-05:00,NaT,NaN,NaN,NaT,NaN,NaN,2020-02-19,2020-02-19
1,Our costs to provide current benefits and fund...,T,https://www.sec.gov/Archives/edgar/data/732717...,Our costs to provide current benefits and fund...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,...,4813 Telephone Communications (No Radiotelephone),2020-02-19 21:42:50-05:00,NaT,NaN,NaN,NaT,NaN,NaN,2020-02-19,2020-02-19


In [15]:
merge2['drop_row1'] = np.where( (merge2['Date_Added'].notna() & (merge2['date_filed'] < merge2['Date_Added'] ) ), 1, 0)

In [16]:
merge2['drop_row2'] = np.where( (merge2['Date_Removed'].notna() & (merge2['date_filed'] > merge2['Date_Removed']) ), 1, 0)

In [ ]:
#merge2[merge2.drop_row1 == 1]
#merge2[merge2.drop_row2 == 1]

In [17]:
print(merge2.shape)

f2_all = merge2[merge2.drop_row1 != 1]
print(f2_all.shape)

f3_all = f2_all[f2_all.drop_row2 != 1]
print(f3_all.shape)

(395167, 26)
(352813, 26)
(332633, 26)


In [18]:
# apply lemmatizer to preprocessed text

lemma = WordNetLemmatizer()  

f3_all['unstemmed'] = f3_all.preproc_text.str.split()

f3_all['stem_text'] = f3_all.unstemmed.apply(lambda x: [lemma.lemmatize(y) for y in x])

f3_all['clean_text'] = f3_all.stem_text.apply(' '.join)

<ipython-input-18-fc7e1a7922f1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f3_all['unstemmed'] = f3_all.preproc_text.str.split()
<ipython-input-18-fc7e1a7922f1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f3_all['stem_text'] = f3_all.unstemmed.apply(lambda x: [lemma.lemmatize(y) for y in x])
<ipython-input-18-fc7e1a7922f1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [19]:
f3_all.head()

,value,ticker,linkToTxt,preproc_text,id,accessionNo,cik,companyName,companyNameLong,formType,...,Date_Removed,Removed_Ticker,Removed_Security,date_filed_str,date_filed,drop_row1,drop_row2,unstemmed,stem_text,clean_text
0,We produce and distribute high-quality video c...,T,https://www.sec.gov/Archives/edgar/data/732717...,We produce and distribute high quality video c...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,...,NaT,NaN,NaN,2020-02-19,2020-02-19,0,0,"[We, produce, and, distribute, high, quality, ...","[We, produce, and, distribute, high, quality, ...",We produce and distribute high quality video c...
1,Our costs to provide current benefits and fund...,T,https://www.sec.gov/Archives/edgar/data/732717...,Our costs to provide current benefits and fund...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,...,NaT,NaN,NaN,2020-02-19,2020-02-19,0,0,"[Our, costs, to, provide, current, benefits, a...","[Our, cost, to, provide, current, benefit, and...",Our cost to provide current benefit and fundin...
2,Our international operations have increased ou...,T,https://www.sec.gov/Archives/edgar/data/732717...,Our international operations have increased ou...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,...,NaT,NaN,NaN,2020-02-19,2020-02-19,0,0,"[Our, international, operations, have, increas...","[Our, international, operation, have, increase...",Our international operation have increased our...
3,"In addition, operating in foreign countries al...",T,https://www.sec.gov/Archives/edgar/data/732717...,In addition operating in foreign countries al...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,...,NaT,NaN,NaN,2020-02-19,2020-02-19,0,0,"[In, addition, operating, in, foreign, countri...","[In, addition, operating, in, foreign, country...",In addition operating in foreign country also ...
4,Our video subsidiaries derive substantial reve...,T,https://www.sec.gov/Archives/edgar/data/732717...,Our video subsidiaries derive substantial reve...,82eece9ad661306bdc4460b8fdea0aee,0001562762-20-000064,732717,AT&T INC.,AT&T INC. (Filer),10-K,...,NaT,NaN,NaN,2020-02-19,2020-02-19,0,0,"[Our, video, subsidiaries, derive, substantial...","[Our, video, subsidiary, derive, substantial, ...",Our video subsidiary derive substantial revenu...


In [20]:
f3_all.columns

Index(['value', 'ticker', 'linkToTxt', 'preproc_text', 'id', 'accessionNo',
       'cik', 'companyName', 'companyNameLong', 'formType', 'filedAt',
       'linkToHtml', 'irsNo', 'fiscalYearEnd', 'sic', 'filed_at', 'Date_Added',
       'Added_Ticker', 'Added_Security', 'Date_Removed', 'Removed_Ticker',
       'Removed_Security', 'date_filed_str', 'date_filed', 'drop_row1',
       'drop_row2', 'unstemmed', 'stem_text', 'clean_text'],
      dtype='object')

In [21]:
f4_all = f3_all[['date_filed', 'filedAt', 'formType', 'accessionNo', 'ticker', 'cik', 
                'companyName', 'companyNameLong', 'linkToTxt', 'formType',
                'sic', 'fiscalYearEnd', 'value', 'preproc_text', 'clean_text']]

In [22]:
with open('../data/preproc/preproc_text_all_years.pickle', 'wb') as to_write:
    pickle.dump(f4_all, to_write)